In [48]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np

#2010

url = "https://www.best-selling-cars.com/britain-uk/2010-britain-best-selling-car-models-in-the-uk/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
ol_element = soup.find('ol')
list_items = ol_element.find_all('li')

data = []
for item in list_items:
    text = item.get_text(strip=True)
    make_model_sold = text.split('…')
    make_model = make_model_sold[0].strip()
    cars_sold = make_model_sold[1].strip()
    
    data.append((make_model, cars_sold, 2010))


#2011
    
url = f"https://www.best-selling-cars.com/britain-uk/2011-full-year-top-ten-best-selling-car-models-in-the-uk/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
ol_element = soup.find('ol')
list_items = ol_element.find_all('li')

for item in list_items:
    text = item.get_text(strip=True)
    make_model_sold = text.split('–')
    make_model = make_model_sold[0].strip()
    cars_sold = make_model_sold[1].strip()
    
    data.append((make_model, cars_sold, 2011))


#2012

url = f"https://www.best-selling-cars.com/britain-uk/2012-full-year-britain-best-selling-car-models-in-the-uk/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
ol_element = soup.find('ol').find('li').find('ol').find('ol')
list_items = ol_element.find_all('li')

for item in list_items:
    text = item.get_text(strip=True)
    make_model_sold = text.split('–')
    make_model = make_model_sold[0].strip()
    cars_sold = make_model_sold[1].strip()
    
    data.append((make_model, cars_sold, 2012))


#2013

url =f"https://www.best-selling-cars.com/britain-uk/2013-full-year-britain-best-selling-car-models-uk/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')
rows = table.find_all('tr')

for row in rows[1:11]:  # Skip the header row
    cols = row.find_all('td')
    brand_model = cols[1].text
    cars_sold = cols[2].text.strip().replace("’", ",")

    data.append((brand_model, cars_sold, 2013))


#2014-16 + 2018

years = list(range(2014, 2017)) + [2018]

for year in (years):

    url =f"https://www.best-selling-cars.com/britain-uk/{year}-full-year-britain-best-selling-car-models/"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows[1:11]:  # Skip the header row
        cols = row.find_all('td')
        brand_model = cols[1].text
        cars_sold = cols[2].text

        data.append((brand_model, cars_sold, str(year)))


#2017
        
driver = webdriver.Firefox()

driver.get("https://www.best-selling-cars.com/britain-uk/2017-full-year-britain-top-ten-best-selling-car-models/")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
rows = table.find_all('tr')


for row in rows[1:11]:
    cols = row.find_all('td')
    brand_model = cols[1].text
    cars_sold = cols[2].text

    data.append((brand_model, cars_sold, 2017))

driver.quit()


#2019-23

years = range(2019, 2024)

for year in (years):
    url =f"https://www.best-selling-cars.com/britain-uk/{year}-full-year-britain-best-selling-car-models-in-the-uk/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows[1:11]:
        cols = row.find_all('td')
        brand_model = cols[1].text
        cars_sold = cols[2].text

        data.append((brand_model, cars_sold, str(year)))

#Some of the make and model names are inconsistent across years
corrections = {
    'Mercedes Benz A Class': 'Mercedes-Benz A Class', 'Mercedes-Benz A-Class': 'Mercedes-Benz A Class',
    'Mercedes C Class': 'Mercedes-Benz C Class', 'MINI': 'MINI Mini', 'Mini': 'MINI Mini'}
corrected_data = [(corrections.get(brand_model, brand_model), cars_sold, year) for brand_model, cars_sold, year in data]

#Create df and pivot to correct format
top_ten_by_year_df = pd.DataFrame(corrected_data, columns = ['model', 'number_sold', 'year'])
pivot_top_ten_by_year_df = top_ten_by_year_df.pivot(index='model', columns='year')
pivot_top_ten_by_year_df.index = pivot_top_ten_by_year_df.index.map(lambda x: ' '.join(x.split()[1:]))
pivot_top_ten_by_year_df.columns = pivot_top_ten_by_year_df.columns.droplevel(0)

#Remove sales figures for Teslas in 2021 and 2022
pivot_top_ten_by_year_df.loc[['Model Y', 'Model 3']] = np.nan

#Change the index label to match rest of code
pivot_top_ten_by_year_df.index.name = 'Model'
pivot_top_ten_by_year_df = pivot_top_ten_by_year_df.apply(lambda x: x.str.replace(',', '').astype('Int64'))
pivot_top_ten_by_year_df.to_csv('sales_data.csv', index_label=None)
print(pivot_top_ten_by_year_df)

year        2010   2011    2012    2013   2017    2014    2015    2016   2018  \
Model                                                                           
A3          <NA>   <NA>    <NA>    <NA>   <NA>   45581   47653   43808   <NA>   
1 Series    <NA>   <NA>   34488   41883   <NA>    <NA>    <NA>    <NA>   <NA>   
3 Series   42020  42471   44521   43494   <NA>    <NA>    <NA>    <NA>   <NA>   
500         <NA>   <NA>    <NA>    <NA>   <NA>   44005    <NA>    <NA>   <NA>   
Fiesta    103013  96112  109265  121929  94533  131254  133434  120525  95892   
Focus      77804  81832   83115   87350  69903   85140   83816   70545  50492   
Kuga        <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>  40398   
Puma        <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>   <NA>   
Tucson      <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>   <NA>   
Sportage    <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>  35567   
Mini       41883  35845    <

In [44]:
print(pivot_top_ten_by_year_df)

year        2010   2011    2012    2013   2017    2014    2015    2016   2018  \
model                                                                           
A3          <NA>   <NA>    <NA>    <NA>   <NA>   45581   47653   43808   <NA>   
1 Series    <NA>   <NA>   34488   41883   <NA>    <NA>    <NA>    <NA>   <NA>   
3 Series   42020  42471   44521   43494   <NA>    <NA>    <NA>    <NA>   <NA>   
500         <NA>   <NA>    <NA>    <NA>   <NA>   44005    <NA>    <NA>   <NA>   
Fiesta    103013  96112  109265  121929  94533  131254  133434  120525  95892   
Focus      77804  81832   83115   87350  69903   85140   83816   70545  50492   
Kuga        <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>  40398   
Puma        <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>   <NA>   
Tucson      <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>   <NA>   
Sportage    <NA>   <NA>    <NA>    <NA>   <NA>    <NA>    <NA>    <NA>  35567   
Mini       41883  35845    <